Imported Libaries

In [1]:
import os
root_dir = r'c:\Users\jf79\Cloned Repos\Footfall-Repo'
os.chdir(root_dir)

import pandas as pd

import myfuncs.myfuncs as mf
import myfuncs.footfallfuncs as ff

mf.read_directory()

import warnings
warnings.filterwarnings('ignore')

Your Current Directory is: c:\Users\jf79\Cloned Repos\Footfall-Repo
Files in: ['.git', 'Footfall.venv', 'Footfall_Scripts', 'myfuncs', 'README.md']



In [2]:
cwd = r'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data'
mf.read_directory(cwd)

Directory being read is: C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data
Files in: ['Area Based', 'Hex Based', 'LSOA Based', 'Mapping']



In [3]:
footfall_2024_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/hex_3hourly_counts_2024.csv')
hex_to_borough_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/BT Hex ID to Borough Lookup Table.csv')
relevant_hexes_data = pd.read_csv(f'{cwd}/Hex Based/Relevant Hexes/Relevant Hexes.csv')
bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')

In [4]:
values_to_keep = [
    "Sands End","Fulham Town","Parsons Green & Sandford","Palace & Hurlingham","Munster",
    "Walham Green","Lillie","Fulham Reach","West Kensington","Hammersmith Broadway",
    "Avonmore","Ravenscourt","Brook Green","Grove","Addison","Wendell Park",
    "Coningham","Shepherd's Bush Green","Wormholt","White City","College Park & Old Oak"
]

bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')
ward_footfall = bespoke_areas_data.copy()

ward_footfall = ward_footfall[ward_footfall['name'].isin(values_to_keep)]
ward_footfall['hours'] = ward_footfall['hours'].str.strip("'")
ward_footfall.rename(
    columns={
        'hours':'time_indicator',
    },
    inplace=True
)
ward_footfall


,bespoke_area_id,name,count_date,time_indicator,resident,visitor,worker,ave_loyalty_percentage,ave_dwell_time
1953166,257,Sands End,2022-05-01,00-03,957,179,89,20.26,109.56
1953167,257,Sands End,2022-05-01,03-06,12538,4633,969,2.16,104.36
1953168,257,Sands End,2022-05-01,06-09,13383,5149,1224,3.42,139.48
1953169,257,Sands End,2022-05-01,09-12,15997,8379,1660,6.97,100.04
1953170,257,Sands End,2022-05-01,12-15,14451,10695,1972,5.95,88.25
...,...,...,...,...,...,...,...,...,...
2122626,277,College Park & Old Oak,2025-02-09,09-12,31155,13968,5345,7.69,87.92
2122627,277,College Park & Old Oak,2025-02-09,12-15,30906,21957,5956,9.36,73.75
2122628,277,College Park & Old Oak,2025-02-09,15-18,30295,23090,6002,8.35,74.16
2122629,277,College Park & Old Oak,2025-02-09,18-21,31449,17293,4932,6.38,83.96


In [5]:
lon_footfall_2024 = footfall_2024_data.copy()
relevant_hexes = relevant_hexes_data.copy()
hex_to_borough = hex_to_borough_data.copy()

borough_footfall_2024 = pd.merge(
    lon_footfall_2024,
    hex_to_borough,
    right_on='Hex_ID',
    left_on='hex_id',
    how='left'
)

borough_footfall_2024 = borough_footfall_2024.groupby(['borough_name','count_date','time_indicator']).agg(
    resident = ('resident','sum'),
    worker = ('worker','sum'),
    visitor = ('visitor','sum')
).reset_index()

hf_footfall_2024 = pd.merge(
    relevant_hexes_data,
    lon_footfall_2024,
    left_on='Hex_ID',
    right_on='hex_id',
    how='left'
)

In [6]:
typical_day_averages = ff.typical_footfall(
    ward_footfall, '2024-01-01', '2024-12-31',
    footfall_type=['residents','workers','visitors'],
    day_night=False,
    primary_key='name'
)

for i in range(len(typical_day_averages)):
    if 'averages' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='averages', ascending=False))
    elif 'daytime_mean' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='daytime_mean', ascending=False))
    else:
        display(typical_day_averages[i])

Calculating typical daily footfall...
For Weedays and Weekends and Weekly averages...


Aggregating footfall data...
Missing kwargs: {'day_night', 'time_indicator'}
These args will be set to default values

Applying features...
Features applied.


Detecting anomalies...
248 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
0 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
216 anomalies have been detected.
Anomalies have been flagged and corrected.

Footfall Data Aggregated.



,year,name,averages
5,2024,Fulham Reach,346370.879586
10,2024,Munster,279509.380952
18,2024,West Kensington,275680.645980
8,2024,Hammersmith Broadway,260960.640515
15,2024,Shepherd's Bush Green,258165.980679
16,2024,Walham Green,255712.030055
11,2024,Palace & Hurlingham,247613.192623
7,2024,Grove,245143.782397
9,2024,Lillie,243123.389539
2,2024,Brook Green,239131.183646


,year,week_name,name,averages
5,2024,Weekday,Fulham Reach,355611.642857
10,2024,Weekday,Munster,280895.379226
18,2024,Weekday,West Kensington,280024.227917
8,2024,Weekday,Hammersmith Broadway,277873.914667
16,2024,Weekday,Walham Green,257823.987186
15,2024,Weekday,Shepherd's Bush Green,256675.561614
7,2024,Weekday,Grove,251840.576336
2,2024,Weekday,Brook Green,251007.698473
11,2024,Weekday,Palace & Hurlingham,249437.139586
9,2024,Weekday,Lillie,244515.551799


,year,week_name,name,averages
26,2024,Weekend,Fulham Reach,323091.264423
31,2024,Weekend,Munster,276017.731456
39,2024,Weekend,West Kensington,264738.160714
36,2024,Weekend,Shepherd's Bush Green,261920.690247
37,2024,Weekend,Walham Green,250391.522665
32,2024,Weekend,Palace & Hurlingham,243018.249313
30,2024,Weekend,Lillie,239616.211538
28,2024,Weekend,Grove,228273.013049
29,2024,Weekend,Hammersmith Broadway,218352.199863
40,2024,Weekend,White City,213814.742445


In [7]:
mf.export_to_csv(typical_day_averages[0])

Exproting typical_ward_averages to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\typical_ward_averages.csv

Successfully exported typical_ward_averages to CSV
